## Liberaries Imports

In [1]:
import face_recognition
from deepface import DeepFace
import json
import os
import cv2
import numpy as np
import os
import pprint

## Static Face Recognition

In [2]:
# Load an image from file
image = face_recognition.load_image_file(".\Images\group\group_image.png")

# Detect all faces in the image
face_locations = face_recognition.face_locations(image)

# Print the number of faces detected
print(f"Found {len(face_locations)} face(s) in this photograph.")

Found 10 face(s) in this photograph.


In [3]:
# Convert the image from RGB (used by face_recognition) to BGR (used by OpenCV)
image_bgr = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

# Draw a rectangle around each face
for (top, right, bottom, left) in face_locations:
    cv2.rectangle(image_bgr, (left, top), (right, bottom), (0, 255, 0), 2)

# Display the result
cv2.imshow('Faces Found', image_bgr)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [4]:
# Get the face encodings for each face
face_encodings = face_recognition.face_encodings(image, face_locations)

In [5]:
known_face_encodings = []
known_face_names = []

# Iterate over the images in the Individuals folder
for filename in os.listdir(".\images\individuals"):
    if filename.endswith(".jpeg"):
        known_image = face_recognition.load_image_file(f".\Images\Individuals\{filename}")
        known_encoding = face_recognition.face_encodings(known_image)[0]
        known_face_encodings.append(known_encoding)
        known_face_names.append(filename.replace(".jpeg", ""))

In [6]:
# Load an image with an unknown face
unknown_image = face_recognition.load_image_file(".\Images\group\group_image.png")

# Find all face locations and face encodings in the unknown image
unknown_face_locations = face_recognition.face_locations(unknown_image)
unknown_face_encodings = face_recognition.face_encodings(unknown_image, unknown_face_locations)

# Loop over each face found in the unknown image
for face_encoding, face_location in zip(unknown_face_encodings, unknown_face_locations):
    
    # See if the face is a match for the known faces
    matches = face_recognition.compare_faces(known_face_encodings, face_encoding)
    
    name = "Unknown"

    # If a match was found, use the known person's name
    if True in matches:
        first_match_index = matches.index(True)
        name = known_face_names[first_match_index]

    # Draw a box around the face
    top, right, bottom, left = face_location
    cv2.rectangle(unknown_image, (left, top), (right, bottom), (0, 255, 0), 2)

    # Label the image with the name
    cv2.putText(unknown_image, name, (left + 6, bottom - 6), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)


# Display the result
cv2.imshow('Face Recognition', unknown_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

## Static Face Attributes detection

In [30]:
final_result = {}

for filename in os.listdir(".\images\individuals"):
    if filename.endswith(".jpeg"):
        known_image = face_recognition.load_image_file(f".\Images\Individuals\{filename}")
        known_image = cv2.cvtColor(known_image, cv2.COLOR_BGR2RGB)
        try:
            detector = DeepFace.analyze(known_image, enforce_detection=False)
            final_result[filename.replace(".jpeg", "")] = detector
        except ValueError as e:
            print(f"Error processing {filename}: {e}")

pprint.pprint(final_result)

Action: race: 100%|██████████| 4/4 [00:00<00:00,  4.92it/s]  

{'Abdallah_Mohamed': [{'age': 23,
                       'dominant_emotion': 'happy',
                       'dominant_gender': 'Man',
                       'dominant_race': 'middle eastern',
                       'emotion': {'angry': 6.378863684444887e-06,
                                   'disgust': 5.045902219280377e-07,
                                   'fear': 0.0025336362942147282,
                                   'happy': 99.31600685811084,
                                   'neutral': 0.6674011170060108,
                                   'sad': 0.013002829702739699,
                                   'surprise': 0.0010477757029248354},
                       'face_confidence': 0.91,
                       'gender': {'Man': 99.99967813491821,
                                  'Woman': 0.00031815122838452226},
                       'race': {'asian': 1.2118784710764885,
                                'black': 0.9098192676901817,
                                'indian': 9

## Real-Time Face Recognition

In [16]:
# Initialize the webcam
video_capture = cv2.VideoCapture(0)

# Loop over each frame from the video until interrupted (e.g., by pressing 'q')
while True:
    # Grab a single frame of video
    ret, frame = video_capture.read()
    if not ret:
        print("Failed to grab frame")
        break

    # Convert the frame from BGR (OpenCV) to RGB (face_recognition)
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Find all face locations and encodings in the frame
    face_locations = face_recognition.face_locations(rgb_frame)
    face_encodings = face_recognition.face_encodings(rgb_frame, face_locations)

    # Loop over each face in the frame
    for face_encoding, face_location in zip(face_encodings, face_locations):
        matches = face_recognition.compare_faces(known_face_encodings, face_encoding)
        name = "Unknown"

        # Check if there is a match and get the corresponding name
        if True in matches:
            first_match_index = matches.index(True)
            name = known_face_names[first_match_index]

        # Draw a rectangle around the face
        top, right, bottom, left = face_location
        cv2.rectangle(frame, (left, top), (right, bottom), (0, 255, 0), 2)

        # Label the face with the name
        cv2.putText(frame, name, (left + 6, bottom - 6), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)

    # Display the video feed
    cv2.imshow('Video', frame)

    # Press 'q' to quit
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the webcam and close the window
video_capture.release()
cv2.destroyAllWindows()

## Real-Time Facial Attribute Detection (Emotion Recognition)

In [19]:
import cv2
from fer import FER
import matplotlib.pyplot as plt

# Initialize the webcam
video_capture = cv2.VideoCapture(0)

# Initialize FER emotion detector
emotion_detector = FER()

while True:
    # Capture frame-by-frame
    ret, frame = video_capture.read()

    # Check if frame is successfully captured
    if not ret:
        print("Failed to grab frame")
        break

    # Detect emotions in the frame
    emotions = emotion_detector.detect_emotions(frame)

    # Loop through detected faces and emotions
    for emotion_data in emotions:
        # Get bounding box (face location) and emotions for each face
        (top, right, bottom, left) = emotion_data['box']
        emotion, score = max(emotion_data['emotions'].items(), key=lambda item: item[1])

        # Draw a rectangle around the face
        cv2.rectangle(frame, (left, top), (right, bottom), (0, 255, 0), 2)

        # Display the emotion and score near the face
        text = f"{emotion}: {score:.2f}"
        cv2.putText(frame, text, (left, top - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 255, 255), 2)

    # Display the resulting frame
    cv2.imshow('Emotion Recognition', frame)

    # Press 'q' to quit
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the webcam and close windows
video_capture.release()
cv2.destroyAllWindows()


## Real-Time Facial Landmarks Detection (DeepFace)

In [ ]:
import cv2
import dlib
from deepface import DeepFace

# Initialize Dlib's face detector and facial landmark predictor
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')  

# Initialize the webcam
video_capture = cv2.VideoCapture(0)

while True:
    # Capture frame-by-frame
    ret, frame = video_capture.read()

    # Detect faces using Dlib
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = detector(gray)

    # Analyze attributes using DeepFace for each detected face
    for face in faces:
        # Get bounding box coordinates for the face
        x, y, w, h = (face.left(), face.top(), face.width(), face.height())

        # Extract the face region from the frame and convert it to an image
        face_image = frame[y:y+h, x:x+w]
        face_image = cv2.cvtColor(face_image, cv2.COLOR_BGR2RGB)

        # Analyze the face with DeepFace to detect age, gender, emotion
        try:
            analysis = DeepFace.analyze(face_image, enforce_detection=False)
            age = analysis[0]['age']
            gender = analysis[0]['dominant_gender']
            emotion = analysis[0]['dominant_emotion']
            # race = analysis[0]['dominant_race']
            
            # Display the detected age, gender, and emotion
            cv2.putText(frame, f'Age: {age}', (x, y - 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
            cv2.putText(frame, f'Gender: {gender}', (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
            cv2.putText(frame, f'Emotion: {emotion}', (x, y + h + 20), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
        except ValueError as e:
            print(f"Error analyzing face: {str(e)}")
            continue

        # Detect and display facial landmarks using Dlib
        landmarks = predictor(gray, face)
        for n in range(0, 68):  # Loop over the 68 facial landmarks
            x_point = landmarks.part(n).x
            y_point = landmarks.part(n).y
            cv2.circle(frame, (x_point, y_point), 2, (0, 255, 0), -1)

        # Draw a rectangle around the face
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

    # Display the video feed
    cv2.imshow('Video', frame)

    # Press 'q' to quit
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the webcam and close the window
video_capture.release()
cv2.destroyAllWindows()


## The Beast MODE
#### (Real-Time Face Recognition with facial landmarks,age,gender and emotions Predictor)

In [20]:
import cv2
import dlib
from deepface import DeepFace
import face_recognition

# Initialize Dlib's face detector and facial landmark predictor
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')

# Initialize the webcam
video_capture = cv2.VideoCapture(0)

while True:
    # Capture frame-by-frame
    ret, frame = video_capture.read()
    if not ret:
        print("Failed to grab frame")
        break

    # Convert the frame to grayscale for Dlib
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # Convert the frame from BGR to RGB for face_recognition
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Detect faces using Dlib
    dlib_faces = detector(gray)

    # Detect faces using face_recognition
    face_locations = face_recognition.face_locations(rgb_frame)
    face_encodings = face_recognition.face_encodings(rgb_frame, face_locations)

    # Process each detected face
    for (top, right, bottom, left), face_encoding in zip(face_locations, face_encodings):
        # Face recognition
        matches = face_recognition.compare_faces(known_face_encodings, face_encoding)
        name = "Unknown"
        if True in matches:
            first_match_index = matches.index(True)
            name = known_face_names[first_match_index]

        # Extract the face region
        face_image = frame[top:bottom, left:right]
        face_image_rgb = cv2.cvtColor(face_image, cv2.COLOR_BGR2RGB)

        # Analyze the face with DeepFace
        try:
            analysis = DeepFace.analyze(face_image_rgb, enforce_detection=False)
            age = analysis[0]['age']
            gender = analysis[0]['dominant_gender']
            emotion = analysis[0]['dominant_emotion']

            # Display results
            cv2.putText(frame, f'Name: {name}', (left, top - 50), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
            cv2.putText(frame, f'Age: {age}', (left, top - 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
            cv2.putText(frame, f'Gender: {gender}', (left, top - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
            cv2.putText(frame, f'Emotion: {emotion}', (left, bottom + 20), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
        except ValueError as e:
            print(f"Error analyzing face: {str(e)}")
            continue

        # Draw a rectangle around the face
        cv2.rectangle(frame, (left, top), (right, bottom), (0, 255, 0), 2)

        # Detect and display facial landmarks using Dlib
        dlib_face = dlib.rectangle(left, top, right, bottom)
        landmarks = predictor(gray, dlib_face)
        for n in range(0, 68):  # Loop over the 68 facial landmarks
            x_point = landmarks.part(n).x
            y_point = landmarks.part(n).y
            cv2.circle(frame, (x_point, y_point), 2, (0, 255, 0), -1)

    # Display the video feed
    cv2.imshow('Video', frame)

    # Press 'q' to quit
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the webcam and close the window
video_capture.release()
cv2.destroyAllWindows()

Action: race: 100%|██████████| 4/4 [00:01<00:00,  3.97it/s]  


-------

WE can make this a functional Function in a ELearning platform to track student learning.
- based on the studednt profile bicture. and name
- generate a report about the student learning session.

### Future Work & Potential Integrations

1. **Attention Mechanism**:
   - You could implement an attention mechanism in your face recognition system that allows the model to focus on specific parts of the input image or sequence of frames. This could enhance the accuracy of facial attribute detection and recognition.
   - Attention can help in interpreting which areas of the face are most relevant when making predictions, possibly improving the results of your facial analysis.

2. **Image Captioning**:
   - Use NLP to generate descriptive captions for images detected in the video feed. For instance, once a face is recognized and analyzed, you could create a textual summary of the analysis results (e.g., "This is John, a 25-year-old male, looking happy.").
   - You could combine visual features extracted from the image with textual descriptions for enhanced data representation.

3. **Sentiment Analysis**:
   - Analyze the emotions detected using DeepFace and combine this with NLP techniques to evaluate sentiment in any text associated with the detected individual. For example, if a recognized person has spoken text (like a transcript), you could analyze how their spoken words relate to their facial expressions.
   - This could be particularly useful in applications like customer service or interactive AI where understanding both visual and textual cues is important.

4. **Using Grad-CAM for Interpretability**:
   - Grad-CAM (Gradient-weighted Class Activation Mapping) can be utilized to visualize which parts of the image influenced the model’s predictions. After facial analysis, you can overlay Grad-CAM heatmaps on the detected faces to show which facial features led to specific attributes or recognition results.
   - This can improve the interpretability of your model, allowing users to understand why certain predictions were made.

5. **Multimodal Data Analysis**:
   - Combine text data (e.g., transcripts, comments) with visual data (video or images) for a more holistic analysis. For example, when analyzing a video, you can detect faces and simultaneously transcribe speech, analyzing how facial expressions align with spoken content.

### Example Workflow

1. **Face Detection and Analysis**:
   - Use the existing face recognition system to detect and analyze faces in real-time.

2. **Text Data Integration**:
   - As faces are recognized, pull any associated textual data (like comments or transcripts) and analyze it using NLP techniques.

3. **Generate Captions**:
   - Use NLP to generate captions or textual descriptions based on the results of facial analysis (age, gender, emotions).

4. **Visualize Attention with Grad-CAM**:
   - Apply Grad-CAM to visualize which facial regions contributed to the attribute predictions, displaying these overlays on the video feed.

--------------